---
# **Exercícios - Classificação Baseada em Exemplos**
---

**Autor**
> Vitor Eduardo de Souza Costa

**Referência**
> Solange Rezende. Paradigma de aprendizado baseado em instâncias. https://edisciplinas.usp.br/pluginfile.php/8366136/mod_resource/content/1/Aula_15_IA_MedidasDistancia_KNN.pdf. Mai. de 2024.

In [ ]:
# @title Importando as bibliotecas necessárias

import pandas as pd, graphviz
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

---
## Exercício 1
---

### Criar dataset de diagnóstico de pacientes

> No exercício proposto os dados dispostos são referentes a 6 diferentes pacientes, comentando sobre os sintomas sentidos por cada e o diagnóstico dado a estes pacientes. O objetivo é utilizar destes dados para rotular novos casos dados e verificados anteriormente utilizando árvores de decisão: https://colab.research.google.com/drive/1kgW38A01mgxexplgXtmm7L5WU0tT0TdX?usp=sharing.

In [ ]:
# @title Criando dataset de diagnóstico

%%writefile cadastro_pacientes_train.tsv
Nome;Febre;Enjôo;Manchas;Dores;Diagnóstico
João;sim;sim;pequenas;sim;doente
Pedro;não;não;grandes;não;saudável
Maria;sim;sim;pequenas;não;saudável
José;sim;não;grandes;sim;doente
Ana;sim;não;pequenas;sim;saudável
Leila;não;não;grandes;sim;doente

Overwriting cadastro_pacientes_train.tsv


In [ ]:
# @title Criando dataset para predição de diagnóstico

%%writefile cadastro_pacientes_classify.tsv
Nome;Febre;Enjôo;Manchas;Dores
Luis;não;não;pequenas;sim
Laura;sim;sim;grandes;sim

Overwriting cadastro_pacientes_classify.tsv


In [ ]:
# @title Lendo dataset de Treino
# Indexa pelo nome, busca predizer a coluna "Diagnóstico"
dataset_diagnostico = pd.read_csv('cadastro_pacientes_train.tsv', index_col='Nome', sep=';')

dataset_diagnostico

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
João,sim,sim,pequenas,sim,doente
Pedro,não,não,grandes,não,saudável
Maria,sim,sim,pequenas,não,saudável
José,sim,não,grandes,sim,doente
Ana,sim,não,pequenas,sim,saudável
Leila,não,não,grandes,sim,doente


In [ ]:
# @title Lendo dataset para Classificar
# Indexa pelo nome, busca predizer a coluna "Diagnóstico"
prev_diagnostico = pd.read_csv('cadastro_pacientes_classify.tsv', index_col='Nome', sep=';')

prev_diagnostico

,Febre,Enjôo,Manchas,Dores
Nome,,,,
Luis,não,não,pequenas,sim
Laura,sim,sim,grandes,sim


### Limpar e Tratar o Dataset

> Com os dados disponíveis, é necessário facilitar seu tratamento computacional convertendo valores simbolicos em numéricos, conservando ordem em caso de valores ordinais e mantendo distância unitária caso não haja distância unitária. Em nosso caso, como é uma base minúscula e sem problemas, esta etapa se resume à conversão dos valores categóricos.

In [ ]:
# @title Convertendo valores simbólicos em numéricos

dataset_diagnostico.Febre.replace({'sim': 1,'não': 0}, inplace=True)
dataset_diagnostico.Enjôo.replace({'sim': 1,'não': 0}, inplace=True)
dataset_diagnostico.Manchas.replace({'grandes': 1,'pequenas': 0}, inplace=True)
dataset_diagnostico.Dores.replace({'sim': 1,'não': 0}, inplace=True)
dataset_diagnostico.Diagnóstico.replace({'saudável': 1,'doente': 0}, inplace=True)

# Criação do dataframe retirando o rótulo
dataframe_diagnostico = dataset_diagnostico.drop(['Diagnóstico'], axis=1)

dataframe_diagnostico

,Febre,Enjôo,Manchas,Dores
Nome,,,,
João,1,1,0,1
Pedro,0,0,1,0
Maria,1,1,0,0
José,1,0,1,1
Ana,1,0,0,1
Leila,0,0,1,1


In [ ]:
# @title Convertendo valores simbólicos em numéricos para exemplos para predição

prev_diagnostico.Febre.replace({'sim': 1,'não': 0}, inplace=True)
prev_diagnostico.Enjôo.replace({'sim': 1,'não': 0}, inplace=True)
prev_diagnostico.Manchas.replace({'grandes': 1,'pequenas': 0}, inplace=True)
prev_diagnostico.Dores.replace({'sim': 1,'não': 0}, inplace=True)

prev_diagnostico

,Febre,Enjôo,Manchas,Dores
Nome,,,,
Luis,0,0,0,1
Laura,1,1,1,1


### Solução para K = 1

> Criação de um algoritmo K-NN com K = 1, para comparar resultados obtidos para diferentes valores de K e exibir possíveis viéses destes modelos.

In [ ]:
# @title Construção do Modelo

k1 = KNeighborsClassifier(n_neighbors=1)

In [ ]:
# @title Obtenção das classes para exemplos

diag_classifier_k1 = k1.fit(dataframe_diagnostico,dataset_diagnostico.Diagnóstico)

diag_classified_k1 = diag_classifier_k1.predict(prev_diagnostico)
diag_results_k1 = prev_diagnostico.copy()
diag_results_k1['Diagnóstico'] = diag_classified_k1

diag_results_k1

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
Luis,0,0,0,1,1
Laura,1,1,1,1,0


In [ ]:
# @title Retornando valores numéricos para categóricos

diag_results_k1.Febre.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k1.Enjôo.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k1.Dores.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k1.Manchas.replace({0: 'Pequenas', 1: 'Grandes'}, inplace=True)
diag_results_k1.Diagnóstico.replace({0: 'Doente', 1: 'Saudável'}, inplace=True)

diag_results_k1

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
Luis,Não,Não,Pequenas,Sim,Saudável
Laura,Sim,Sim,Grandes,Sim,Doente


### Solução para K = 3

> Criação de um algoritmo K-NN com K = 3, para comparar resultados obtidos para diferentes valores de K e exibir possíveis viéses destes modelos.

In [ ]:
# @title Construção do Modelo

k3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# @title Obtenção das classes para exemplos

diag_classifier_k3 = k3.fit(dataframe_diagnostico,dataset_diagnostico.Diagnóstico)
diag_classified_k3 = diag_classifier_k3.predict(prev_diagnostico)
diag_results_k3 = prev_diagnostico.copy()
diag_results_k3['Diagnóstico'] = diag_classified_k3

diag_results_k3

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
Luis,0,0,0,1,0
Laura,1,1,1,1,0


In [ ]:
# @title Retornando valores numéricos para categóricos

diag_results_k3.Febre.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k3.Enjôo.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k3.Dores.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k3.Manchas.replace({0: 'Pequenas', 1: 'Grandes'}, inplace=True)
diag_results_k3.Diagnóstico.replace({0: 'Doente', 1: 'Saudável'}, inplace=True)

diag_results_k3

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
Luis,Não,Não,Pequenas,Sim,Doente
Laura,Sim,Sim,Grandes,Sim,Doente


### Solução para K = 5

> Criação de um algoritmo K-NN com K = 5, para comparar resultados obtidos para diferentes valores de K e exibir possíveis viéses destes modelos.

In [ ]:
# @title Construção do Modelo

k5 = KNeighborsClassifier(n_neighbors=5)

In [ ]:
# @title Obtenção das classes para exemplos

diag_classifier_k5 = k5.fit(dataframe_diagnostico,dataset_diagnostico.Diagnóstico)
diag_classified_k5 = diag_classifier_k5.predict(prev_diagnostico)
diag_results_k5 = prev_diagnostico.copy()
diag_results_k5['Diagnóstico'] = diag_classified_k5

diag_results_k5

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
Luis,0,0,0,1,0
Laura,1,1,1,1,0


In [ ]:
# @title Retornando valores numéricos para categóricos

diag_results_k5.Febre.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k5.Enjôo.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k5.Dores.replace({0: 'Não', 1: 'Sim'}, inplace=True)
diag_results_k5.Manchas.replace({0: 'Pequenas', 1: 'Grandes'}, inplace=True)
diag_results_k5.Diagnóstico.replace({0: 'Doente', 1: 'Saudável'}, inplace=True)

diag_results_k5

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
Luis,Não,Não,Pequenas,Sim,Doente
Laura,Sim,Sim,Grandes,Sim,Doente


---
## Exercício 2
---

### Criar dataset de classe e escolaridade

> No exercício proposto são disponibilizados dados referentes a escolaridade, estado e renda de diferentes pessoas. Busca-se registrar os dados disponibilizados no exercício para implementar uma solução de classificação.

In [ ]:
# @title Criando Dataset

%%writefile classe_escolaridade.tsv
Estado;Escolaridade;Altura;Salário;Classe
SP;Médio;180;3000;A
RJ;Superior;174;7000;B
RS;Médio;180;600;B
RJ;Superior;100;2000;A
SP;Fundamental;178;5000;A
RJ;Fundamental;188;1800;A


Overwriting classe_escolaridade.tsv


In [ ]:
# @title Criando dataset com dados novos para serem classificados

%%writefile classificar_escolaridade.tsv
Estado;Escolaridade;Altura;Salário
RJ;Médio;178;2000
SP;Superior;200;800

Overwriting classificar_escolaridade.tsv


In [ ]:
# @title Lendo dataset
# Indexa pelo nome, busca predizer a coluna "Diagnóstico"
dataset = pd.read_csv('classe_escolaridade.tsv', sep=';')

dataset

,Estado,Escolaridade,Altura,Salário,Classe
0,SP,Médio,180,3000,A
1,RJ,Superior,174,7000,B
2,RS,Médio,180,600,B
3,RJ,Superior,100,2000,A
4,SP,Fundamental,178,5000,A
5,RJ,Fundamental,188,1800,A


In [ ]:
# @title Lendo dataset com valores a classificar

rank = pd.read_csv('classificar_escolaridade.tsv', sep=';')

rank

,Estado,Escolaridade,Altura,Salário
0,RJ,Médio,178,2000
1,SP,Superior,200,800


### Limpar e tratar o dataset

> Com os dados disponíveis, é necessário facilitar seu tratamento computacional convertendo valores simbolicos em numéricos, conservando ordem em caso de valores ordinais e mantendo distância unitária caso não haja distância unitária. Em nosso caso, como é uma base minúscula e sem problemas, esta etapa se resume à conversão dos valores categóricos.

In [ ]:
# @title Convertendo valores simbólicos em numéricos

#Valores simbólicos com distância unitária ou ordinais com ordem definida
dataset.Classe.replace({'A': 1,'B': 0}, inplace=True)
dataset.Escolaridade.replace({'Superior': 2, 'Médio': 1, 'Fundamental': 0}, inplace=True)

#Tratametno para manter distância unitária em valores não ordinais
onehot = pd.get_dummies(dataset.Estado,dtype=int)
dataset = pd.concat([onehot, dataset.drop('Estado', axis=1)], axis=1)

dataframe = dataset.drop('Classe', axis=1)
dataframe


,RJ,RS,SP,Escolaridade,Altura,Salário
0,0,0,1,1,180,3000
1,1,0,0,2,174,7000
2,0,1,0,1,180,600
3,1,0,0,2,100,2000
4,0,0,1,0,178,5000
5,1,0,0,0,188,1800


In [ ]:
# @title Convertendo valores simbólicos em numéricos para valores a classificar

#Valores simbólicos com distância unitária ou ordinais com ordem definida
rank.Escolaridade.replace({'Superior': 2, 'Médio': 1, 'Fundamental': 0}, inplace=True)

#Tratametno para manter distância unitária em valores não ordinais
onehot_rank = pd.get_dummies(rank.Estado,dtype=int)
rank = pd.concat([onehot_rank, rank.drop('Estado', axis=1)], axis=1)

#Recuperando coluna perdida para o dado chegar da mesma forma que os presentes no treinamento
RS = [0]*len(rank.RJ)
RS_pd = pd.DataFrame(RS,columns=['RS'])

rank = pd.concat([rank.RJ,RS_pd,rank.drop('RJ', axis=1)],axis=1)
rank

,RJ,RS,SP,Escolaridade,Altura,Salário
0,1,0,0,1,178,2000
1,0,0,1,2,200,800


### Solução para K = 1

> Criação de um algoritmo K-NN com K = 1, para comparar resultados obtidos para diferentes valores de K e exibir possíveis viéses destes modelos.

In [ ]:
# @title Construção do Modelo

k1 = KNeighborsClassifier(n_neighbors=1)

In [ ]:
# @title Obtenção das classes para exemplos

classifier_k1 = k1.fit(dataframe,dataset.Classe)

ranked_k1 = classifier_k1.predict(rank)
ranked_k1 = pd.DataFrame(ranked_k1,columns=['Classe'])
results_k1 = pd.concat([rank,ranked_k1],axis=1)

In [ ]:
# @title Retornando valores numéricos para categóricos

results_k1.Escolaridade.replace({1:'Médio',2:'Superior'}, inplace=True)
results_k1.Classe.replace({1:'A',0:'B'}, inplace=True)
dummies_k1 = pd.concat([results_k1.RJ, results_k1.RS, results_k1.SP], axis=1)
onecold_k1 = pd.from_dummies(dummies_k1)
onecold_k1.rename(columns={'':'Estado'}, inplace=True)
results_k1 = pd.concat([onecold_k1, results_k1.drop(['RJ','RS','SP'], axis=1)], axis=1)

results_k1

,Estado,Escolaridade,Altura,Salário,Classe
0,RJ,Médio,178,2000,A
1,SP,Superior,200,800,B


> Para possível verificação destes resultados foi produzida uma planilha disponível na pasta de arquivos anexados a este *notebook*, nomeada "*exercício_2.xlsx*". Os resultados apresentados são condizentes com os preditos na planilha para o caso de K = 1.

### Solução para K = 3

> Criação de um algoritmo K-NN com K = 3, para comparar resultados obtidos para diferentes valores de K e exibir possíveis viéses destes modelos.

In [ ]:
# @title Construção do Modelo

k3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# @title Obtenção das classes para exemplos

classifier_k3 = k3.fit(dataframe,dataset.Classe)

ranked_k3 = classifier_k3.predict(rank)
ranked_k3 = pd.DataFrame(ranked_k3,columns=['Classe'])
results_k3 = pd.concat([rank,ranked_k3],axis=1)

In [ ]:
# @title Retornando valores numéricos para categóricos

results_k3.Escolaridade.replace({1:'Médio',2:'Superior'}, inplace=True)
results_k3.Classe.replace({1:'A',0:'B'}, inplace=True)
dummies_k3 = pd.concat([results_k3.RJ, results_k3.RS, results_k3.SP], axis=1)
onecold_k3 = pd.from_dummies(dummies_k3)
onecold_k3.rename(columns={'':'Estado'}, inplace=True)
results_k3 = pd.concat([onecold_k3, results_k3.drop(['RJ','RS','SP'], axis=1)], axis=1)

results_k3

,Estado,Escolaridade,Altura,Salário,Classe
0,RJ,Médio,178,2000,A
1,SP,Superior,200,800,A


> Agora, com os resultados para K = 3, conseguimos também confirmar na planilha exercício_2.xlsx que os valores de classe para ambos os testes condiz com o esperado.

###Calculando a precisão do algoritmo